# Data cleaning and exploration

This script takes as input the raw data from the dataset `SQL_aggregated_data.csv`, which was generated using the SQL script `SQL_script.sql`. This dataset is then re-processed to aggregate over those retailers active in both the trial and future period. The output file `Reshaped_dataset.csv` contains a single entry per retailer containing the necessary trial and future period data.

In [ ]:
# import libraries
import os
import pandas as pd

In [ ]:
# read in the dataset from the SQL script output
df = pd.read_csv('SQL_aggregated_data.csv', header = 0, )

In [ ]:
# check number of rows and columns
df.shape

In [ ]:
# see variable names
df.keys()

In [ ]:
# take only those retailers that appear in both the trial and future period
both_periods = df['CustomAttribute1'][df.CustomAttribute1.duplicated()]
df_tmp = df[df.CustomAttribute1.isin(both_periods)]

In [ ]:
df_tmp.shape

In [ ]:
# remove unneeded columns
df_tmp = df_tmp.drop(columns = ['Profit','total_conversionvalue'])

In [ ]:
# pull out all features for trial period and drop period column and rename some columns
aggr_trial_features = df_tmp[df_tmp['period'] == 'trial'].drop(columns = 'period')
aggr_trial_features = aggr_trial_features.rename(columns = {'anyRevenue': 'anyRevenue_trial', 'Revenue':'Revenue_trial'})

In [ ]:
aggr_trial_features.shape

In [ ]:
# pull out relevant features for future period and rename columns
aggr_future_profit = df_tmp[df_tmp['period'] == 'future'][['CustomAttribute1','anyRevenue','Revenue']].rename(columns = {'anyRevenue': 'anyRevenue_future', 'Revenue':'Revenue_future'})

In [ ]:
# merge trial and future data on retailer
MergedFile = pd.merge(aggr_trial_features,aggr_future_profit,on = 'CustomAttribute1')

In [ ]:
# save the resulting merged aggregated data for the ML script
MergedFile.to_csv("Reshaped_dataset.csv", index=False, encoding='utf-8-sig')